# Working with Drug Indications

This notebook shows how to get drug indications from ChEMBL. Note that on your first run, the download step will take longer depending on your internet connection.

In [1]:
import sys
import time

import pandas as pd
from tqdm.auto import tqdm

from chembl_downloader import get_date, latest, queries, query

In [2]:
print(sys.version)

3.11.0 (main, Oct 25 2022, 14:13:24) [Clang 14.0.0 (clang-1400.0.29.202)]


In [3]:
print(time.asctime())

Fri Oct 28 11:55:25 2022


## Building a SQL Query

In this example, we're going to re-use a common query from the `chembl_downloader.queries` module.

The following SQL query is used to combine the `DRUG_INDICATION` table with extra information from the `MOLECULE_DICTIONARY` table using join. This guarantees that we only get rows about molecules that have indications.

In [4]:
queries.markdown(queries.DRUG_INDICATIONS_SQL)

```sql
SELECT
    MOLECULE_DICTIONARY.chembl_id,
    MOLECULE_DICTIONARY.pref_name,
    MOLECULE_DICTIONARY.chebi_par_id,
    DRUG_INDICATION.mesh_id,
    DRUG_INDICATION.mesh_heading,
    DRUG_INDICATION.efo_id AS indication_curie,
    DRUG_INDICATION.efo_term AS indication_label,
    DRUG_INDICATION.max_phase_for_ind
FROM MOLECULE_DICTIONARY
JOIN DRUG_INDICATION ON MOLECULE_DICTIONARY.molregno == DRUG_INDICATION.molregno
```

## Running the SQL Query

We're first going to explicitly get the latest ChEMBL version so we can document it in the notebook, then execute the query using `chembl_downloader.query` to get a Pandas DataFrame.

In [5]:
latest_version = latest()

print(f"Using latest ChEMBL version: {latest_version}")

Using latest ChEMBL version: 31


In [6]:
%time
df = query(queries.DRUG_INDICATIONS_SQL, version=latest_version)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.91 µs


In [7]:
df

,chembl_id,pref_name,chebi_par_id,mesh_id,mesh_heading,indication_curie,indication_label,max_phase_for_ind
0,CHEMBL1201823,ABATACEPT,NaN,D045743,"Scleroderma, Diffuse",EFO:0000404,diffuse scleroderma,2
1,CHEMBL1201823,ABATACEPT,NaN,D001172,"Arthritis, Rheumatoid",EFO:0000685,rheumatoid arthritis,4
2,CHEMBL1201823,ABATACEPT,NaN,D008180,"Lupus Erythematosus, Systemic",EFO:0002690,systemic lupus erythematosus,2
3,CHEMBL1201584,ABCIXIMAB,NaN,D009203,Myocardial Infarction,EFO:0000612,myocardial infarction,3
4,CHEMBL271227,ABIRATERONE ACETATE,68639.0,D011471,Prostatic Neoplasms,EFO:0001663,prostate carcinoma,4
...,...,...,...,...,...,...,...,...
48811,CHEMBL245807,OXYPHENISATINE,NaN,D003248,Constipation,HP:0002019,Constipation,4
48812,CHEMBL4802233,ROXATIDINE,NaN,D005764,Gastroesophageal Reflux,EFO:0003948,gastroesophageal reflux disease,0
48813,CHEMBL4802233,ROXATIDINE,NaN,D010437,Peptic Ulcer,HP:0004398,Peptic ulcer,0
48814,CHEMBL4802223,LEUPROLIDE MESYLATE,NaN,D011471,Prostatic Neoplasms,MONDO:0008315,prostate cancer,4


## Comparison

We'll take the query of the drug indications to the next level by comparing results across ChEMBL versions. We chose a minimum version of 26 to show a bit of variety, but not eat up too much disk space, since each version is around 20-25GB.

In [8]:
rows = []
it = tqdm(
    list(reversed(range(26, int(latest_version) + 1))),
    unit="version",
    leave=False,
)
for version in it:
    version = str(version)
    it.set_postfix(version=version)
    df = query(queries.DRUG_INDICATIONS_SQL, version=version)
    rows.append(
        (
            version,
            get_date(version),
            len(df.index),
            len(df["chembl_id"].unique()),
            len(df["mesh_id"].unique()),
        )
    )

  0%|          | 0/6 [00:00<?, ?version/s]

In [9]:
counts_df = pd.DataFrame(
    rows,
    columns=["version", "date", "associations", "unique_molecules", "unique_indications"],
)
counts_df

,version,date,associations,unique_molecules,unique_indications
0,31,2022-07-12,48816,8074,1888
1,30,2022-02-22,48816,8074,1888
2,29,2021-07-01,45902,7792,1781
3,28,2021-01-15,42988,7446,1730
4,27,2020-05-18,37259,6700,1523
5,26,2020-02-14,37259,6700,1523
